# ウイルス感染症の数理シミュレーション（応用）: 補足資料

Julia言語で書いている

In [1]:
using DifferentialEquations
using ModelingToolkit
using Plots

# B型感染ウイルスの数理モデリング

## 解離・接着までのモデル

In [2]:
let
    @independent_variables t
    @variables V_dis(t)=1.0 V_att(t)=150.0
    @parameters k_dis=3.4 k_att=0.16
    
    D = Differential(t)
    eqs = [
        D(V_dis) ~ k_dis * V_att - k_att * V_dis
        D(V_att) ~ k_att * V_dis - k_dis * V_att
    ]
    @mtkbuild sys = ODESystem(eqs, t)
    
    tspan = (0.0, 30.0)
    prob = ODEProblem(sys, [], tspan)
    sol = solve(prob)
    
    dinamics_plot = plot(
        sol[t],
        [sol[V_dis], sol[V_att]],
        title="Virus dinamics",
        xlabel="Hour",
        ylabel="Virus",
        label=["dissociate" "attach"],
        color=[:blue :orange],
    )
        
    plot(
        dinamics_plot,
        size=(600, 300),
        left_margin = 20Plots.px,
        bottom_margin = 20Plots.px,
        legend=:outertopright,
    )
end

## 全状態を考慮したモデル

In [3]:
function virus_dinamics(
    v::Dict{String, Float64},
    p::Dict{String, Float64},
    t_end::Float64=70.0 / 24.0,
)
    @independent_variables t
    @variables V_dis(t)=v["V_dis"]
    @variables V_att(t)=v["V_att"]
    @variables V_int(t)=v["V_int"]
    @variables V_ccc(t)=v["V_ccc"]
    @parameters k_dis=p["k_dis"]
    @parameters k_att=p["k_att"]
    @parameters k_int=p["k_int"]
    @parameters k_ccc=p["k_ccc"]
    @parameters k_di=p["k_di"]
    @parameters k_dc=p["k_dc"]
    @parameters K=p["K"]
    
    D = Differential(t)
    eqs = [
        D(V_dis) ~ k_dis * V_att - k_att * V_dis
        D(V_att) ~ k_att * V_dis - k_dis * V_att - k_int * V_att
        D(V_int) ~ k_int * V_att - k_ccc * (1 - V_ccc / K) * V_int - k_di * V_int
        D(V_ccc) ~ k_ccc * (1 - V_ccc / K) * V_int - k_dc * V_ccc
    ]
    @mtkbuild sys = ODESystem(eqs, t)
    
    tspan = (0.0, t_end)
    prob = ODEProblem(sys, [], tspan)
    return solve(prob)
end

virus_dinamics (generic function with 2 methods)

In [4]:
let
    v::Dict{String, Float64} = Dict(
        "V_dis" => 1.0,
        "V_att" => 150.0,
        "V_int" => 1.0,
        "V_ccc" => 1.0,
    )
    p::Dict{String, Float64} = Dict(
        "k_dis" => 3.4,
        "k_att" => 0.16,
        "k_int" => 3.0,
        "k_ccc" => 0.65,
        "k_di" => 1.9,
        "k_dc" => 0.12,
        "K" => 3.7,
    )
    sol = virus_dinamics(v, p)
    u = reduce(hcat, sol.u)
    
    dinamics_plot = plot(
        sol.t,
        [u[1, :] u[2, :] u[3, :] u[4, :]],
        title="Virus dinamics",
        xlabel="Day",
        ylabel="Virus",
        label=["dissociate" "attach" "internalization" "cccDNA"],
        color=[:blue :orange :red :firebrick],
    )
    cccDNA_plot = plot(
        sol.t,
        u[4, :],
        title="cccDNA dinamics",
        xlabel="Day",
        ylabel="cccDNA",
        label="cccDNA",
        color=:firebrick,
    )
        
    plot(
        dinamics_plot,
        cccDNA_plot,
        size=(900, 300),
        left_margin = 20Plots.px,
        bottom_margin = 20Plots.px,
    )
end

In [5]:
let
    v::Dict{String, Float64} = Dict(
        "V_dis" => 1.0,
        "V_att" => 150.0,
        "V_int" => 1.0,
        "V_ccc" => 1.0,
    )
    p::Dict{String, Float64} = Dict(
        "k_dis" => 3.4,
        "k_att" => 0.16,
        "k_int" => 3.0,
        "k_ccc" => 0.65,
        "k_di" => 1.9,
        "k_dc" => 0.12,
        "K" => 3.7,
    )
    
    inhibition_rate = Vector(0:0.01:1)
    
    u_att::Vector{Float64} = Vector()
    u_int::Vector{Float64} = Vector()
    u_ccc::Vector{Float64} = Vector()
    
    for alpha = inhibition_rate
        p["k_att"] = (1 - alpha) * 0.16
        sol = virus_dinamics(v, p, 12.0)
        u = reduce(hcat, sol.u)
        push!(u_att, u[4, end])
    end
    p["k_att"] = 0.16
    
    for alpha = inhibition_rate
        p["k_int"] = (1 - alpha) * 3.0
        sol = virus_dinamics(v, p, 12.0)
        u = reduce(hcat, sol.u)
        push!(u_int, u[4, end])
    end
    p["k_int"] = 3.0
    
    for alpha = inhibition_rate
        p["k_ccc"] = (1 - alpha) * 0.65
        sol = virus_dinamics(v, p, 12.0)
        u = reduce(hcat, sol.u)
        push!(u_ccc, u[4, end])
    end
    p["k_ccc"] = 0.65
    
    cccDNA_plot = plot(
        inhibition_rate,
        [u_att u_int u_ccc],
        title="Inhibition rate effect",
        xlabel="Inhibition rate",
        ylabel="cccDNA (after 12 days)",
        label=["attach" "internalization" "cccDNA"],
    )
        
    plot(
        cccDNA_plot,
        size=(600, 300),
        left_margin = 20Plots.px,
        bottom_margin = 20Plots.px,
        legend=:outertopright,
    )
end